# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [4]:
# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.
    

In [23]:
#1 Ajustando o modelo log(renda)

#1.1 Utilizando o Patsy
formula = 'np.log(renda) ~ posse_de_veiculo + posse_de_imovel + qtd_filhos + C(sexo) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia + 1'

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())


                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 15 Dec 2023   Prob (F-statistic):               0.00
Time:                        23:50:18   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [24]:
# Verificando quais serão minhas caselas de referência:

# Contagem de frequência das categorias
frequency_counts = df.apply(lambda x: x.value_counts().idxmax())

print(frequency_counts)

Unnamed: 0                         0
data_ref                  2015-01-01
id_cliente                      9948
sexo                               F
posse_de_veiculo               False
posse_de_imovel                 True
qtd_filhos                         0
tipo_renda               Assalariado
educacao                  Secundário
estado_civil                  Casado
tipo_residencia                 Casa
idade                             40
tempo_emprego               4.216438
qt_pessoas_residencia            2.0
renda                        5402.44
dtype: object


In [25]:
# Suponha que 'sexo', 'tipo_renda', 'educacao', 'estado_civil' e 'tipo_residencia' sejam variáveis categóricas
formula = 'np.log(renda) ~ posse_de_veiculo + posse_de_imovel + qtd_filhos + C(sexo, Treatment(reference="F")) + C(tipo_renda, Treatment(reference="Assalariado")) + C(educacao, Treatment(reference="Secundário")) + C(estado_civil, Treatment(reference="Casado")) + C(tipo_residencia, Treatment(reference="Casa")) + idade + tempo_emprego + qt_pessoas_residencia'

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 15 Dec 2023   Prob (F-statistic):               0.00
Time:                        23:50:35   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [26]:
# Suponha que 'sexo', 'tipo_renda', 'educacao', 'estado_civil' e 'tipo_residencia' sejam variáveis categóricas
formula = 'np.log(renda) ~ posse_de_veiculo + posse_de_imovel + qtd_filhos + C(sexo, Treatment(reference="F")) + C(tipo_residencia, Treatment(reference="Casa")) + idade + tempo_emprego + qt_pessoas_residencia'

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     549.1
Date:                Fri, 15 Dec 2023   Prob (F-statistic):               0.00
Time:                        23:50:40   Log-Likelihood:                -13670.
No. Observations:               12427   AIC:                         2.737e+04
Df Residuals:                   12414   BIC:                         2.746e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

In [29]:
# Suponha que 'sexo', 'tipo_renda', 'educacao', 'estado_civil' e 'tipo_residencia' sejam variáveis categóricas
formula = 'np.log(renda) ~ posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego'

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     621.1
Date:                Fri, 15 Dec 2023   Prob (F-statistic):               0.00
Time:                        23:56:23   Log-Likelihood:                -14930.
No. Observations:               12427   AIC:                         2.987e+04
Df Residuals:                   12421   BIC:                         2.992e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

Tanto o R² quanto o R² ajustado diminuiram, o que pode indicar que o modelo piorou.